<h1>CreateSaveModelAmorphousML.ipynb</h1> 

This code requires that AmorphousFileLectureCreate.ipynb has been run. If you are confident that the model structures are correct (go to TestAFolder to test the structures) then this code creates the model. It trains on ALL experiments that were processed with AmorphousFileLectureCreate.ipynb 

__________________________________________________________________________________________

OUTPUTS OF THE CODE: 

1. **AmorphousLogFileModelCreation.txt**
A log file with every step that the algorithm has followed


2. **AmorphousExecution_times.txt**
It times how long the code took to create each model (0.5-1 mins aprox. per model)


3. **ModelsAmorphous**
For each Complexity and each num_augmentations (a.k.a, for each model structure and data organization) a folder is created.

    3.1 **Model\_{Complexity}\_{num\_augmentations}**
**THESE ARE THE FOLDERS YOU NEED TO COPY AND PASTE INSIDE THE PREDICTING CODE FOLDERS. THE WHOLE FOLDER NOT JUST THE CONTENTS**
Inside these folders you have the entire model and the scalers all ready to use for prediction. 
Note: The corrections are dependent on the static parameters and not the model or the training routine. Therefore this models will not give you the corrected predictions. (Do not worry, in the prediction files these corrections are automatically done if you don't change the flag that activates them)
 
       3.1.1 Model_{Complexity}_{num_augmentations}.keras The model file

       3.1.2 scaler_static_{Complexity}_{num_augmentations}_PolarizationD3.pkl. The scaler for the static parameters (initial and final polarizations included) used in this isolated-experiment iteration

       3.1.3 scaler_time_{Complexity}_{num_augmentations}_PolarizationD3.pkl. The scaler for the time evolution used in this isolated-experiment iteration

       3.1.4 scaler_y_{Complexity}_{num_augmentations}.pkl. The scaler for the polarization values used in this isolated-experiment iteration


__________________________________________________________________________________________

Process:

It takes two lists o
By changing the lists Names and Augmentations you can pick what model you create. Just write on the two lists your model names and the number of augmentations and it will create models combining them 
As an example,

Names = ["ModelA","ModelB","ModelC"]

Augmentations = [5,9,2]

will create the models "ModelA_5","ModelB_9" and "ModelC_2"

In order to do that, the code first prepares the folder structure and the folder where the model will be stored.
It then loops (if told to) over all polarization columns to create models for both.
It extracts all information from the data base, scales it and saves those scalers
Finally it trains the model and saves it in a distinct folder. That folder contains the scalers (in order to decipher how to scale it back to physical units) and the trained model.

Here are some explanations of what the code does in a more detiled manner:

1. **load\_experiments**. It uses the directory where the array (numerical values) and parameter files resides, picks one polarization column and prepares a list of experiments to feed the ML algorythm. The output is as follows:

    *encoded\_experiments=[(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 1}$,(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 2}$,...]* 
    
    Note that the Cell Id is Hot Encoded. The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to Cell_ID. As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes 0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical variables to ML.
    The size of the output list depends on the number of pairs of .txt files present in the folder. This means that it also works for isolated experiments.
    
2. **augment_experiments** is a function that takes the original data list (as load\_experiments returns it) and augments them _num\_augmentations_ times. The final legth of the list will be (num\_augmentations +1) times the original length. Due to the reduced size of the data base (Not more than 30 experiments before 2026), techniques like augmentation are required. Due to the stochastic nature of neutron detection, most measurements, if repeated under the same conditions, will yield different results. Of course, all measurements converge (with uncertainty) to what we can say is "the true value". Therefore, we can duplicate the experiments adding noise to the measurements to obtain "hypothetical" measurements that expand the data base. It was decided that the uncertainty won't be modified. To decide what type of noise could be applied two possibilities were considered. The first one was to suppose that sensor detection follows a Poisson distribution (law of rare events). The second one was to suppose that it follows a Normal Distribution of mean the measured value and width the uncertainty. As a Poisson distribution converges stochastically (in probability and distribution but not almost sure nor in $L^p$) to a normal distribution, it is safe to assume that they converge to the same result so the gaussian approach was selected. Also, the measurements are not raw counts but a function of them.
$P=\frac{n^+-n^-}{n^++n^-}$
We have no direct data of the number of counts (the parameter of a Posissonian distribution) but a rough estimate points to them being (worst case scenario) within the order of the millions (note that after 100 counts, poisson distributions vary very little from normal distributions). Threfore, count detection can be approximated to a normal distribution and a linear combination of random variables distributed as normal distributions is also a random variable with a normal distribution. Therefore, it is safe to assume that sensor measurements can be modelled after a gaussian distribution.


3. **build_dataset** is a function that prepares the data base to be fed directly into a ML model for training and validation. There are a few import decisions taken here. The way this function is set up, it removes 2 rows of data per polariser cell studied. The reason why it was done is because we want the ML model to be able to predict polarization decay when given the specs of the cell (the static parameters) and the initial and final polarization values (with their associated time values). The reason why those two values are considered "known values" for each cell is beacuse they can be easily measured experimentally and they give as a good estimate about the overall behaviour. In some experiments, the environment of the studied sample is too fragile to move and place the Si crystal for polarization measurements. Therefore, a working ML algorithm whose inputs are the specs of the cell and the initial and final polarizations (measured before and after the sample is in place) would enable experiments that could not be done before without proper polarization efficiency corrections. 
Also, we remove those two values per polariser cell from the training arrays (Xt, y and err). The reason why it is done is to avoid data leaks in the model. If we give those values as training data and also give them as parameters, the ML algorithm can run into the risk of memorizing those pairs (over-fitting) and worsen any new predicitions. Uncertainties for the first and last polarization measurements are not added to the static features. This was a decision taken to avoid giving too much weight to two variables that don't have value on their own (they compliment the polarization values but if the ML architecture is as shallow as
the one used here, they can be considered independent variables and reduce the weight and importance of the other variables). 
Another consideration taken here was that the static features get duplicated in Xs a lot of times. One could think that using a similar method that the one used for augmentations of the data sets could also diversify the data base. However, we wanted all measurements of a same session and cell to be coherent 
and it wouldn't make sense to have different static features. Therefore, the safest approach was to only duplicate these values. For the augmented experiments the only parameters that are changed are the initial and final times and polarizations. There is no incompatibility here to what we have just said as these work as "hypothetical independent experiments". This is why augmentation is done before this function gets used.
         

4. **nll_loss** ML algorythms require a way to tell the algorythm if it is learning or not. The most standard practice is with a Loss function. If the loss value goes down that means that the algorythm is learning and, if a step increases the loss, then it is punished and tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative log-likelihood of a normal distribution 
NLL$=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value. Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough estimate on what the uncertainty of a set predictions looks like.
Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector of $N$ predicted values. It can be considered as a random vector of variance:
$Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but using squared values punished big discrepancies in a stronger way).

$StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.

Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model returns then is closer to a poorly calculated linear fit
Therefore, an addition penalty was added.

$OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
where $C$ is a constant used to control the weight of this penalty

5. **Define_Complexity** It consists of a single function called _Define\_Complexity_. Given the name of a model it defines the model function of the ML algorithm. To be precise, it defines a function called _build_model_ every time _Define\_Complexity_ runs. If _Define\_Complexity_ gets run another time, it will define (possibly) another _build_model_ if the _Complexity_ variable changes

Here we have functions that train, validate and fit the models. Some models require the variables to be scaled or will scale them. Extra precautions need to be taken into account

6. **model\_fitting**. It is a function that logs and runs model.fit() on a two-input Keras model and returns the training history. It needs **static, time and polarization variables scaled**. Training is not done using the uncertainties of the data as it was decided that uncertainty information is encoded in the augmentations. Note: No validation is done anywhere in the code. Here are some of the reasons:
    
    6.1. The data base is very small. The amorphous data base contains only 199 points while the crystalline one contains 251. Removing a small percentage of those points for validation might leave the data base too small and underfitting might worsen the result more than fine tuning parametrs with validation.
    
    6.2. A randomized validation split may be physically wrong. Therefore it should be chronological, not shuffled. However, in crystalline experiments, there are decay experiments that have only four or five intermediate points. Even removing one point for validation is a massive hit on the experiment. Therefore, it is risky to add validation
    
    6.3. To find good models, a Leave-one-out approach was used. For a certain model structure, an experiment gets removed and the model and it trains on all the remaining experiments. Then, the model tries to predict this isolated experiment. Afterwards, the experiment is returned and a new one becomes isolated. This process loops for all experiments and an overall score of the model is computed. This process was done for 498 models for crystalline materials. This is a stronger (and more expensive) method than validation as it is not dependent on the validation splits and avoids possible information leaks.

Also, eight randomly picked models were tested with and without validation and with and without an asymetric uncertainty-overestimated penalizing loss. The result showed that the Loss update was an improvement and validation did not increase performance (without validation, the results were slightly better).

7. **model\_prediction**. It is a funtion that predicts with a given model. It needs **static and time variables scaled**. This scaling must be coherent to the one done in the rest of the funtions.

8. **train**. This function is the one responsible of scaling the inputs and training the model (it uses **model\_fitting**)

    8.1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
    
    8.2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized. The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
    
    8.3. It builds the model depending on the use\_uncertainty bool. (It changes the loss function and the output).
    
    8.4. It trains the model and returns its history (the trained model)
    
9. **align_static_vectors**. It converts the columns not present on an isolated experiment to zeros.
    
10. **model_predict_sloped** It substracts a linear function to the predicted values. If done correctly this makes it so that the polarization predictions at the initial and final time points are the same as the measured polarization values at those times. This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still "universal". However we can't fully say that it is a pure ML algorithm. The "correctness" of this method is subjective. It is a warning in the ML front that there is an issue with the data base but it is a valid fix for experimentalists.


## 1. Libraries

In [1]:
import os
import glob
import math
import shutil
import gc
import joblib
import time
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras import Input, Model, regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

## 2. Auxiliary Functions and log file creation

1. _PrintDebug_ is a flag that allows the code to output on screen all the steps. If it is set to false, it won´t show anything. However, all information will be properly logged whether this flag is set to true or false. The name of the log is determined by the variable *log_file_path*. The code runs faster if it is set to False.

2. _ShowPlot_ is a similar flag that allows the code to show on screen all plots that are being produced. They are all stored independently of whether this flag is True or False. The code runs faster if it is set to False.

3. _LogNoise_ is another flag that allows numerical values in the log. Most of these values are not worth keeping but if you want to see if there are no NaNs or zeros you can turn it on


3. **log_message** is a function used for writting on the log file

4. **win_long_path** is a function that "fixes" directory paths

In [2]:
PrintDebug = True #This Bool will determine if all logs should be printed on screen on the Python Notebook. The log writing is always on. If False the code will be faster.
ShowPlot = False #This Bool works the same but with showing on screen the plots (they are always saved even with this variable being False). Reduces program cost if False
LogNoise = False #This Bool allows numerical values in the log. Most of these values are not worth keeping but if you want to see if there are no NaNs or zeros you can turn it on

log_file_path = "AmorphousLogFileModelCreation.txt"
def log_message(message):
    """
    Arguments: 
        message (string): The text that will be logged
    
    Returns:
        None
        
    Notes:
        It will write the string "message" in the log file.
        If PrintDebug==True then it will also print the string
    """
    message = str(message)
    if PrintDebug:
        print(message)
    with open(log_file_path, 'a', encoding='utf-8') as log_file:
        log_file.write(str(message) + "\n")
        
################################################################

def long_path(path):
    """
    Arguments: 
        path (path): The path that needs to be converted
    
    Returns:
        The updated path string or path depending on the platform used
        
    Notes:
        To avoid Windows 260 character limit for Windows paths, a special "prefix" is added.
        It also unifies how directories are managed.
        Also works with Linux and Mac

    """
    # Convert to Path and resolve to absolute
    path = Path(path).resolve()
    
    #Windows only:  
    if os.name == "nt":
        path_str = str(path)
        if not path_str.startswith("\\\\?\\"):
            # UNC paths need special handling
            if path_str.startswith("\\\\"):
                path_str = "\\\\?\\UNC\\" + path_str[2:]
            else:
                path_str = "\\\\?\\" + path_str
            return path_str
    
    return path

to_erase = [
    "AmorphousLogFileModelCreation.txt",
    "AmorphousExecution_times.txt",
    "AmorphousModels"]

for item in to_erase:
    path = os.path.abspath(item)
    if os.path.exists(path):
        try:
            if os.path.isfile(path):
                os.remove(path)
                log_message(f"Deleted file: {path}")
            elif os.path.isdir(path):
                shutil.rmtree(path)
                log_message(f"Deleted folder: {path}")
        except Exception as e:
            log_message(f" Could not delete {path}: {e}")
    else:
        log_message(f"Not found (skipped): {path}")
        
with open(log_file_path, 'w', encoding='utf-8') as log_file:
    log_file.write("=== Log started ===\n")
    log_file.write("All outputs from functions have a string at the beginning to show the origin:\n")

Not found (skipped): C:\Users\gopeb\Desktop\D3MLPolarization-final\D3MLPolarization-main\CreateModels\ML\AmorphousLogFileModelCreation.txt
Not found (skipped): C:\Users\gopeb\Desktop\D3MLPolarization-final\D3MLPolarization-main\CreateModels\ML\AmorphousExecution_times.txt
Not found (skipped): C:\Users\gopeb\Desktop\D3MLPolarization-final\D3MLPolarization-main\CreateModels\ML\AmorphousModels


## 3. Functions

1. **load\_experiments**. It uses the directory where the array (numerical values) and parameter files resides, picks one polarization column and prepares a list of experiments to feed the ML algorythm. The output is as follows:

    *encoded\_experiments=[(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 1}$,(static\_values, Deltatime, PolarizationD3, ErrPolarizationD3)$_{experiment 2}$,...]* 
    
    Note that the Cell Id is Hot Encoded. The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to Cell_ID. As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes 0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical variables to ML.
    The size of the output list depends on the number of pairs of .txt files present in the folder. This means that it also works for isolated experiments.
    
2. **augment_experiments** is a function that takes the original data list (as load\_experiments returns it) and augments them _num\_augmentations_ times. The final legth of the list will be (num\_augmentations +1) times the original length. Due to the reduced size of the data base (Not more than 30 experiments before 2026), techniques like augmentation are required. Due to the stochastic nature of neutron detection, most measurements, if repeated under the same conditions, will yield different results. Of course, all measurements converge (with uncertainty) to what we can say is "the true value". Therefore, we can duplicate the experiments adding noise to the measurements to obtain "hypothetical" measurements that expand the data base. It was decided that the uncertainty won't be modified. To decide what type of noise could be applied two possibilities were considered. The first one was to suppose that sensor detection follows a Poisson distribution (law of rare events). The second one was to suppose that it follows a Normal Distribution of mean the measured value and width the uncertainty. As a Poisson distribution converges stochastically (in probability and distribution but not almost sure nor in $L^p$) to a normal distribution, it is safe to assume that they converge to the same result so the gaussian approach was selected. Also, the measurements are not raw counts but a function of them.
$P=\frac{n^+-n^-}{n^++n^-}$
We have no direct data of the number of counts (the parameter of a Posissonian distribution) but a rough estimate points to them being (worst case scenario) within the order of the millions (note that after 100 counts, poisson distributions vary very little from normal distributions). Threfore, count detection can be approximated to a normal distribution and a linear combination of random variables distributed as normal distributions is also a random variable with a normal distribution. Therefore, it is safe to assume that sensor measurements can be modelled after a gaussian distribution.


3. **build_dataset** is a function that prepares the data base to be fed directly into a ML model for training and validation. There are a few import decisions taken here. The way this function is set up, it removes 2 rows of data per polariser cell studied. The reason why it was done is because we want the ML model to be able to predict polarization decay when given the specs of the cell (the static parameters) and the initial and final polarization values (with their associated time values). The reason why those two values are considered "known values" for each cell is beacuse they can be easily measured experimentally and they give as a good estimate about the overall behaviour. In some experiments, the environment of the studied sample is too fragile to move and place the Si crystal for polarization measurements. Therefore, a working ML algorithm whose inputs are the specs of the cell and the initial and final polarizations (measured before and after the sample is in place) would enable experiments that could not be done before without proper polarization efficiency corrections. 
Also, we remove those two values per polariser cell from the training arrays (Xt, y and err). The reason why it is done is to avoid data leaks in the model. If we give those values as training data and also give them as parameters, the ML algorithm can run into the risk of memorizing those pairs (over-fitting) and worsen any new predicitions. Uncertainties for the first and last polarization measurements are not added to the static features. This was a decision taken to avoid giving too much weight to two variables that don't have value on their own (they compliment the polarization values but if the ML architecture is as shallow as
the one used here, they can be considered independent variables and reduce the weight and importance of the other variables). 
Another consideration taken here was that the static features get duplicated in Xs a lot of times. One could think that using a similar method that the one used for augmentations of the data sets could also diversify the data base. However, we wanted all measurements of a same session and cell to be coherent 
and it wouldn't make sense to have different static features. Therefore, the safest approach was to only duplicate these values. For the augmented experiments the only parameters that are changed are the initial and final times and polarizations. There is no incompatibility here to what we have just said as these work as "hypothetical independent experiments". This is why augmentation is done before this function gets used.
         

4. **nll_loss** ML algorythms require a way to tell the algorythm if it is learning or not. The most standard practice is with a Loss function. If the loss value goes down that means that the algorythm is learning and, if a step increases the loss, then it is punished and tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative log-likelihood of a normal distribution 
NLL$=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value. Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough estimate on what the uncertainty of a set predictions looks like.
Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector of $N$ predicted values. It can be considered as a random vector of variance:
$Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but using squared values punished big discrepancies in a stronger way).

$StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.

Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model returns then is closer to a poorly calculated linear fit
Therefore, an addition penalty was added.

$OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
where $C$ is a constant used to control the weight of this penalty

In [3]:
def load_all_experiments(data_dir, polarization_column):
    """
    Arguments:
        data_dir (str): The direction to the folder where all files to be loaded will be found
        polarization_column: It is the name in the header (in the .txt files) of the column that will be read.
                             It can be 'SoftPolarizationD3' or 'PolarizationD3'. All results were obtained with 'PolarizationD3'
    Output: 
        A list, for all polarization decays like:
        encoded_experiments = [
            (static_values,   # list of static parameters (per experiment)
            Deltatime,       # 1D numpy array of time values
            polarization,    # 1D numpy array of polarization values
            Uncertainty      # 1D numpy array of uncertainty values), ...]
        A pd object with the Hot encoded Ids and the rest of the parameters per experiment (each experiment in a different row)
    
    Notes:
    The steps the code does are the following:
    1. Finds all array files (the ones with the numerical values of the decay) and loops over all of them
    2. For each array files it reconstructs the name of the parameter file. It concatenates all parameter files into one pd structure.
    3. Finally it loops over all array files appending the parameters, time, polarization and uncertainty of every experiment to a common list
    """
    
    log_message(f"    load_experiments: Finding all Array Files...")
    # Step 1:

    arrays_files = sorted(
        glob.glob(os.path.join(data_dir, "*.txt"))) #Find all files that are .txt
    arrays_files = [f for f in arrays_files if not f.endswith("_Parameters.txt")] #Keep only the Arrays (not the parameters)

    encoded_experiments = []
    all_static_df = []
    static_columns = ['CellID','AnalyserID', 'PolariserPressure','AnalyserPressure', 'LabPolarization', 'LabTime'] #Parameter header
    for arrays_path in arrays_files:
        base = os.path.basename(arrays_path)
        # Build parameters filename by adding _Parameters before .txt
        name_without_ext = os.path.splitext(base)[0]
        parameters_filename = f"{name_without_ext}_Parameters.txt"
        parameters_path = os.path.join(data_dir, parameters_filename)

        # Read parameters file
        try:
            parameters_df = pd.read_csv(parameters_path) #Import the parameter file
            if LogNoise:
                log_message(f"    load_experiments: Reading parameters file: {parameters_filename}") #Clutter logging

            #Get the first row as static data
            static_row = parameters_df.iloc[0][static_columns] #Get the parameter numerical values
            all_static_df.append(static_row)

        except Exception as e:
            log_message(f"    ****load_experiments: Failed to read parameters file: {parameters_filename}, error: {e}")
            continue

    log_message(f"    load_experiments: Create combined DataFrame for static parameters...")
    static_df = pd.DataFrame(all_static_df) #Combine all static rows into a Dataframe

    log_message(f"    load_experiments: Collected static data:")
    """
    The type of cell did not affect greatly the predictions. However in the future, it may be useful to give more information to the Cell_IDs.
    As of 2025, these strings are Hot Encoded which means that the code finds all the different types, creates columns for each type and writes
    0 or 1 (bool) depending of whether the cell was from one type or the other. This is the standard procedure to feed categorical varaibles to ML.
    """
    
    log_message(f"    load_experiments: Static dataframe columns:, {static_df.columns.tolist()}")
    log_message(f"    load_experiments: Static dataframe shape:, {static_df.shape}")

    log_message(f"    load_experiments: Hot encoding CellID.")
    categorical_cols = ['CellID', 'AnalyserID']
    static_df = pd.get_dummies(static_df, columns=categorical_cols, prefix=['CellID', 'AnalyserID'])
    # Now second pass: read arrays and create encoded_experiments with encoded static params
    for i, arrays_path in enumerate(arrays_files):
        base = os.path.basename(arrays_path)
        name_without_ext = os.path.splitext(base)[0]
        parameters_filename = f"{name_without_ext}_Parameters.txt"
        parameters_path = os.path.join(data_dir, parameters_filename)
        # log_message(f"Reading arrays file: {base}") #Clutter log
        arrays_df = pd.read_csv(arrays_path) # Reads the time series data 
    
        static_values = static_df.iloc[i].to_list() #Fetches the static parameters corresponding to this experiment
    
        Deltatime = arrays_df["DeltaTime"].values
        polarization = arrays_df[polarization_column].values #Extracts the time array and the selected polarization column.
        #Save the uncertainty even if it is not used afterwards
        Uncertainty = arrays_df["ErrPolarizationD3"].values
        #if len(Deltatime) > 2:
        encoded_experiments.append((static_values, Deltatime, polarization, Uncertainty))
        # log_message(f"Creating Encoded Experiments (appending parameters, time array, polarization array and uncertainty array)") #Clutter log
    log_message(f"    load_experiments: Loaded {len(encoded_experiments)} experiments.")
    return encoded_experiments, static_df.columns.tolist()


#################################################################################################

def augment_experiments(original_experiments, num_augmentations=5, base_seed=42):
    """
    Arguments:
        original_experiments (list): A list, for all polarization decays like encoded_experiments in last function:
            original_experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...]
        num_augmentations (int): The number of augmentations done per experiment.
        base_seed (int): A seed that guarantees reproducibility
    
    Outputs:
        A list that concatenates the original experiments with the augmented ones.
        Instead of being a list of len(original_experiments) it becomes a list of
        len(original_experiments) * (1+ num_augmentations)
    
    Notes: Due to the stochastic nature of neutron detection, most measurements, if
    repeated under the same conditions, will yield different results. Of course, all
    measurements converge (with uncertainty) to what we can say is "the true value".
    Therefore, we can duplicate the experiments adding noise to the measurements to obtain
    "hypothetical" measurements that expand the data base. It was decided that the
    uncertainty won't be modified. To decide what type of noise could be applied two
    possibilities were considered. The first one was to suppose that sensor detection
    follows a Poisson distribution (law of rare events). The second one was to suppose
    that it follows a Normal Distribution of mean the measured value and width the uncertainty.
    As a poissonian distribution converges stochastically (in probability and distribution
    but not almost sure nor in L^p) to a normal distribution, it is safe to assume that they
    converge to the same result so the gaussian approach was selected. Also, the measurements
    are not raw counts but a function of them. P=(n^+ - n^-)/(n^+ + n^-)
    We have no direct data of the number of counts (the parameter of a Posissonian distribution)
    but a rough estimate points to them being (worst case scenario) within the order of the millions
    (After 100 counts, poisson distributions vary very little from normal distributions).
    Threfore, count detection can be approximated to a normal distribution and a linear
    combination of random variables distributed as normal distributions is also a random variable
    with a normal distribution. Therefore, it is safe to assume that sensor measurements can be
    modelled after a gaussian distribution.  
    """
    
    log_message(f"    augment_experiments: Augmenting {len(original_experiments)} a number of {num_augmentations} times")
    augmented_experiments = []
    for idx, (static, time, polar, uncertainty) in enumerate(original_experiments):
        for n in range(num_augmentations):
            seed = hash((idx, n, base_seed)) % 2**32
            rng = np.random.default_rng(seed)
            noise = rng.normal(loc=0.0, scale=uncertainty)
            new_polar = polar + noise
            if LogNoise:
                log_message(f"    augment_experiments:       Augmented experiment {idx} #{n} with seed {seed}")
            augmented_experiments.append((static, time, new_polar, uncertainty))
    log_message(f"    augment_experiments: Augmented to {len(original_experiments + augmented_experiments)} experiments")
    return original_experiments + augmented_experiments


#######################################################################################3

def build_dataset(experiments, mode="PolarizationD3"):
    """
    Arguments:
        experiments (list): It is a list like the output of load_experiments or augment_experiments
            experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...]
        mode (str): The column of polarization that will be used. It can be either 'SoftPolarizationD3' or 'PolarizationD3'.
    
    Output:
        1. Xs. An array of lists. Shape (number_of_samples, number_static_features) Each list contains all the original static parameters (hot encoded) plus the first and last polarization values with uncertainty.
           To be precise, they get added in this order: static parameters + initial time + initial polarization + final time + final polarization.
           For all samples, the static features get added to this list. This means that, if an experiment has M measurements, two get added to the parameter
           list and then those parameter features get written M-2 times in this array
        2. Xt. An array of time. Shape (number_of_samples, 1). All time values that are not used as parameters get added to this array. However they are not
           saved directly. If they are for example 0,120,250 then they get saved as [0],[120],[250]. The reason is compatibility with Keras/TensorFlow.
        3. y. An array of polarization. It is the same as Xt but with polarization values (the type of polarization is determined by _mode_)
        4. err. An array of polarization uncertainties. It is the same as Xt and y but with polarization uncertainties.
    Notes:
        If there are only two rows then the file gets skipped. It shouldn't happen but there is logic for it. The reason why it gets skipped is because
        there would not be any values left to use for training or validation    
    """

    Xs, Xt, y, u = [], [], [], []
    log_message(f"    build_dataset: Starting build_dataset for column {mode} ")
    log_message(f"    build_dataset: Number of experiments to process: {len(experiments)} (should be (num_augmentations+1)*(number_of_experiments-1)")
    
    for exp_idx, (static_params, delta_time, polarization, uncertainty) in enumerate(experiments):
        if len(delta_time) < 2:
            log_message(f"    ****build_dataset:       Skipping experiment {exp_idx}: too few data points (len={len(delta_time)})")
            continue
            

        
        if LogNoise:
            log_message(f"    build_dataset: Adding First and Last Polarization (with time) values as static parameters") #Clutter log
        init_idx = 0
        final_idx = -1
        initial_dt, initial_p = delta_time[init_idx], polarization[init_idx]
        final_dt, final_p = delta_time[final_idx], polarization[final_idx]

        static_vector = static_params + [
            initial_dt, initial_p,
            final_dt, final_p
        ]
        if LogNoise:
            log_message(f"    build_dataset: Experiment {exp_idx}: static_vector length={len(static_vector)} (should be 10 (three parameters, CellID hot encoded creates three posibilities, four for the initial and final polarization) ") #Clutter log
        if LogNoise:
            log_message(f"    build_dataset: Building samples Static+time+polarization") #Clutter log
        
        for t, p, err in zip(delta_time[1:-1], polarization[1:-1], uncertainty[1:-1]): 
            Xs.append(static_vector)
            Xt.append([t])
            y.append(p)
            u.append(err) #We will ignore always uncertainty in parameters and even if they are not used, we will keep uncertainties in the data sets(same dimensions everywhere)

    log_message(f"    build_dataset: Number of experiments processed for mode '{mode}': {len(experiments)}")
    log_message(f"    build_dataset: Final dataset shapes: Xs: {np.array(Xs).shape}, Xt: {np.array(Xt).shape}, y: {np.array(y).reshape(-1, 1).shape}")
    return np.array(Xs), np.array(Xt), np.array(y).reshape(-1, 1), np.array(u).reshape(-1, 1)


#################################################################################

def split_experiments(experiments, val_fraction=0.2, seed=42):
    # Tests of validation. Currently unused
    rng = np.random.default_rng(seed)
    n_exp = len(experiments)
    indices = rng.permutation(n_exp)

    n_val = int(val_fraction * n_exp)
    val_idx = indices[:n_val]
    train_idx = indices[n_val:]

    train_experiments = [experiments[i] for i in train_idx]
    val_experiments   = [experiments[i] for i in val_idx]

    log_message(f"Split experiments: {len(train_experiments)} train / {len(val_experiments)} val")
    return train_experiments, val_experiments

#################################################################################

def nll_loss(y_true, y_pred):
    """
    Arguments:
        y_true (array): An array of the real values.
        y_pred (array): An array of the predicted values
    Output:
     A scalar tensorflow tensor ( () ) with the value of the loss as the the mean Gaussian negative log-likelihood over the batch
    Notes:
        This loss function is not just a pure Negative Loss Likelyhood (NLL).
        ML algorythms require a way to tell the algorythm if it is learning or not. The most standard practice is with a Loss function.
        If the loss value goes down that means that the algorythm is learning and, if a step increases the loss, then it is punished and
        tries other approaches. When using uncertainties when teaching the model, the most common loss function is the NLL or Negative
        log-likelihood of a normal distribution 
        
        NLL$=\frac{1}{2}$log$(σ^2)+\frac{(y−μ)^2}{2σ^2}$
        
        where $\sigma$ is the uncertainty in the predictions, $y$ is the measured value and $\mu$ the predicted value.
        Instead of predicting $σ^2$ directly, we obtain its logarithm to have a more stable process (and avoid accounting precision as
        $\sigma^{-2}$ which is numerically unstable when uncertainties are low).

        However we want to avoid uncertainties that drift too far from the overall model predictions. To achieve that, we can get a rough
        estimate on what the uncertainty of a set predictions looks like. Let $\vec{\mu}=\left(\mu_1,\ldots,\mu_N\right)^T$ be the vector
        of $N$ predicted values. It can be considered as a random vector of variance:
        
        $Var(\vec{\mu})=\frac{1}{N}\sum_{i=1}^N\left(\mu_i-E(\vec{\mu})\right)$
        
        where $E(\vec{\mu})$ is the mean of the predicted values. We then have two different variances, one obtained as the sparseness of
        the predictions, (denoted as $Var\left(\vec{\mu}\right)$, and one obtained as a result of the internal ML calculations (denoted as
        $\sigma^2$). A penalty can be added to the loss functions to force the model to try to reduce this differences. An easy way to model
        it is to obtain the difference of those variances and square the result (taking the absolute value was also a good estimate, but
        using squared values punished big discrepancies in a stronger way).

        $StrayPenalty = B \cdot \left[\log\left(\sigma^2\right)-\log\left(\mathrm{Var}\left(\vec{\mu}\right)\right)\right]^2$
        
        where $B$ is a constant used to control the weight of this penalty. The reason why $Var\left(\vec{\mu}\right)$ was used and not
        $Var\left(\vec{y}\right)$ (with $\vec{y}$ the vector of measured values) was to avoid noise in the original data to tamper with the
        loss function. It would be physically clearer to use measured values sparseness as a way to guide the model but some experimental
        uncertainties are clearly underestimated and that would cause this penalty to dominate the loss and obscure the main loss protocol, the NLL.

        Also, we also want to punish the model if it tries overestimating $\sigma$. If the model is unable to minimize $y-\mu$, in order to
        lower NLL, it increases $\sigma$. If no precautions are taken, this "escape solution" achieves bad predictions with inflated
        uncertainties that simulate a low loss value. A new penalty was added that punishes overestimation of the uncertainties more than
        underestimation (which never happened). The slope correction done further on the pipeline can "fix" this issue but what the model
        returns then is closer to a poorly calculated linear fit. Therefore, an addition penalty was added.

        $OverestimatePenalty= C \cdot \max\left(0, \log\left(\sigma^2\right)-\log\left(\mathrm{Var}(\vec{\mu})\right)\right)$
        
        where $C$ is a constant used to control the weight of this penalty
    """

    mu = y_pred[:, 0:1]
    log_var = y_pred[:, 1:2]

    # Base Gaussian NLL
    precision = tf.exp(-log_var)
    nll = tf.reduce_mean(0.5 * (log_var + tf.square(y_true - mu) * precision))


    mu_centered = mu - tf.reduce_mean(mu)
    sigma_ref = tf.sqrt(tf.reduce_mean(tf.square(mu_centered)) + 1e-6)
    Stray_penalizer = 1e-2 #Parameter used to limit how far the uncertainty predictions stray from the experimental data
    OverUncert_penalizer  = 5e-3 #Parameter used for punishing overestimated uncertainties
    log_var_prior = tf.math.log(sigma_ref ** 2) #Order of magnitude of what uncertainties should look like

    # Prior penalty. It penalizes if uncertainty strays too much from the experimental value
    #It avoids underestimation of uncertainty and overestimation
    penalty_stray = Stray_penalizer * tf.reduce_mean(tf.square(log_var - log_var_prior))

    # Asymmetric penalty: punish overestimation more than underestimation
    penalty_overestimate = OverUncert_penalizer * tf.reduce_mean(tf.nn.relu(log_var - log_var_prior))

    return nll + penalty_stray + penalty_overestimate            

## 4. Model Architecture

It consists of a single function called _Define\_Complexity_. Given the name of a model it defines the model function of the ML algorithm. To be precise, it defines a function called _build_model_ every time _Define\_Complexity_ runs. If _Define\_Complexity_ gets run another time, it will define (possibly) another _build_model_ if the _Complexity_ variable changes

In [4]:
def Define_Complexity(Complexity):
    from tensorflow.keras import Input, Model, regularizers
    from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout, Concatenate
    from tensorflow.keras.optimizers import Adam

    if Complexity == "Average":       
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh')(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model

    elif Complexity == "Complex":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.3)(x_time)
            x_time = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = Dropout(0.3)(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(64, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.3)(x)
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-4))(x)
            x = Dropout(0.2)(x)
            x = Dense(16, activation='tanh')(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model

    elif Complexity == "Simple":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.3)(x_time)
            x_time = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-2))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model

    elif Complexity == "Naif":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif834":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif838":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif854":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif858":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif8532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8104":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif8108":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif81016":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif81032":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        

 





        
    elif Complexity == "Naif1634":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif1638":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif1654":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif1658":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif16532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16104":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif16108":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif161016":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif161032":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
 









    elif Complexity == "Naif2434":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif2438":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif2454":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif2458":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif24532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24104":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif24108":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
    elif Complexity == "Naif241016":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model        
    elif Complexity == "Naif241032":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=24, kernel_size=10, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='relu')(x)

            mu = Dense(1)(x)
            if use_uncertainty:
                log_var = Dense(1)(x)
                output = Concatenate()([mu, log_var])
            else:
                output = mu

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
        
        
        
        
        
        
    elif Complexity == "NaifTwice1D883316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D883332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D883516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D883532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D885316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D885332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D885516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D885532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D843316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D843332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D843516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D843532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D845316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D845332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D845516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D845532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    

        
        
        
        
    elif Complexity == "NaifTwice1D483316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D483332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D483516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D483532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D485316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D485332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D485516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D485532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D443316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D443332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D443516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D443532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwice1D445316":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D445332":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "NaifTwice1D445516":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "NaifTwice1D445532":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=5, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)
            x_time = Dropout(0.1)(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
        
        
        
        
        
    elif Complexity == "NaifTwiceDense414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                    
    elif Complexity == "NaifTwiceDense4116":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense4124":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense8116":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense8124":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense1614":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense1618":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense16116":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense16124":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense2414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense2418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense24116":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense24124":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model

        
        
        
        
    elif Complexity == "NaifTwiceDense424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                    
    elif Complexity == "NaifTwiceDense4216":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense4224":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense8216":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense8224":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense1624":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model    
    elif Complexity == "NaifTwiceDense1628":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense16216":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense16224":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense2424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense2428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense24216":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(16, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "NaifTwiceDense24224":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(24, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)   # moved here
            x = Dense(24, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model
        
        
        
        
    elif Complexity == "SimpleNoDropout444":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "SimpleNoDropout448":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
        
    elif Complexity == "SimpleNoDropout484":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout488":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout4164":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout4168":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          

    elif Complexity == "SimpleNoDropout844":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "SimpleNoDropout848":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "SimpleNoDropout884":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout888":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout8164":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout8168":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model         
    elif Complexity == "SimpleNoDropout1644":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model            
    elif Complexity == "SimpleNoDropout1648":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model  
    elif Complexity == "SimpleNoDropout1684":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout1688":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout16164":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model          
    elif Complexity == "SimpleNoDropout16168":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Conv1D(filters=16, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model         
        
        

        
        
        
    elif Complexity == "Simple414414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple414418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple414424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple414428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple414814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple414818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple414824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple414828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                 
    elif Complexity == "Simple418414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple418418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple418424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple418428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple418814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple418818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple418824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple418828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model   
    elif Complexity == "Simple424414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple424418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple424424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple424428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple424814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple424818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple424824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple424828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                 
    elif Complexity == "Simple428414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple428418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple428424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple428428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple428814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple428818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple428824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple428828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model   
        
        
        
        
        
    elif Complexity == "Simple814414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple814418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple814424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple814428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple814814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple814818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple814824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple814828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                 
    elif Complexity == "Simple818414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple818418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple818424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple818428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple818814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple818818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple818824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple818828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.1)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model   
    elif Complexity == "Simple824414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple824418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple824424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple824428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple824814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple824818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple824824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple824828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=4, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model                 
    elif Complexity == "Simple828414":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple828418":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple828424":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple828428":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(4, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
        
    elif Complexity == "Simple828814":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple828818":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.1)(x)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model 
    elif Complexity == "Simple828824":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x)
            x = Dense(4, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model             
    elif Complexity == "Simple828828":
        def build_model(input_dim_static, use_uncertainty=False):
            static_input = Input(shape=(input_dim_static,), name="static_input")
            time_input = Input(shape=(None, 1), name="time_input")

            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(time_input)
            x_time = Dropout(0.2)(x_time)
            x_time = Conv1D(filters=8, kernel_size=3, activation='relu', padding='same')(x_time)
            x_time = GlobalAveragePooling1D()(x_time)

            x = Concatenate()([static_input, x_time])
            x = Dense(8, activation='tanh', kernel_regularizer=regularizers.l2(1e-3))(x)
            x = Dropout(0.2)(x_time)
            x = Dense(8, activation='tanh')(x)  # second dense layer

            mu = Dense(1)(x)
            log_var = Dense(1)(x)
            output = Concatenate()([mu, log_var])

            model = Model(inputs=[static_input, time_input], outputs=output)
            model.compile(optimizer=Adam(), loss=nll_loss if use_uncertainty else 'mse')
            return model         
        
    else:
        log_message(f"****Define_Complexity: UNKNOWN MODEL TYPE. FAIL INCOMING! Please check that the model name actually exists.")
    log_message(f"Define_Complexity: Model built\n")
    return build_model

## 5. Model specific functions

Here we have functions that train, validate and fit the models. Some models require the variables to be scaled or will scale them. Extra precautions need to be taken into account

1. **model\_fitting**. It is a function that logs and runs model.fit() on a two-input Keras model and returns the training history. It needs **static, time and polarization variables scaled**. Training is not done using the uncertainties of the data as it was decided that uncertainty information is encoded in the augmentations. Note: No validation is done anywhere in the code. Here are some of the reasons:
    
    1.1. The data base is very small. The amorphous data base contains only 199 points while the crystalline one contains 251. Removing a small percentage of those points for validation might leave the data base too small and underfitting might worsen the result more than fine tuning parametrs with validation.
    
    1.2. A randomized validation split may be physically wrong. Therefore it should be chronological, not shuffled. However, in crystalline experiments, there are decay experiments that have only four or five intermediate points. Even removing one point for validation is a massive hit on the experiment. Therefore, it is risky to add validation
    
    1.3. To find good models, a Leave-one-out approach was used. For a certain model structure, an experiment gets removed and the model and it trains on all the remaining experiments. Then, the model tries to predict this isolated experiment. Afterwards, the experiment is returned and a new one becomes isolated. This process loops for all experiments and an overall score of the model is computed. This process was done for 498 models for crystalline materials. This is a stronger (and more expensive) method than validation as it is not dependent on the validation splits and avoids possible information leaks.

Also, eight randomly picked models were tested with and without validation and with and without an asymetric uncertainty-overestimated penalizing loss. The result showed that the Loss update was an improvement and validation did not increase performance (without validation, the results were slightly better).

2. **model\_prediction**. It is a funtion that predicts with a given model. It needs **static and time variables scaled**. This scaling must be coherent to the one done in the rest of the funtions.

3. **train**. This function is the one responsible of scaling the inputs and training the model (it uses **model\_fitting**)

     3.1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
    
     3.2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized. The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
    
     3.3. It builds the model depending on the use\_uncertainty bool. (It changes the loss function and the output).
    
     3.4. It trains the model and returns its history (the trained model)
    
4. **align_static_vectors**. It converts the columns not present on an isolated experiment to zeros.
    
5. **model_predict_sloped** It substracts a linear function to the predicted values. If done correctly this makes it so that the polarization predictions at the initial and final time points are the same as the measured polarization values at those times. This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still "universal". However we can't fully say that it is a pure ML algorithm. The "correctness" of this method is subjective. It is a warning in the ML front that there is an issue with the data base but it is a valid fix for experimentalists.

In [5]:
def model_fitting(model, X_static_scaled, X_time_scaled, y_scaled, epochs, batch_size, verbose, callbacks=None):
    """
    Arguments: 
        model (keras model): This is the object that the build_model function returns.
        X_static_scaled (array): It is an array os size (total number of intermediate measured points (let's call them 'samples'), number of static features). 
                                 Using the variables form build_dataset, the number of static features is 4+len(static_parameters(hot encoded)).
                                 To be precise, it is just Xs scaled (an array of lists with the static features) but reshpaed into a 2d array
        X_time_scaled (array): A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        y_scaled (array): The same as X_time_scaled but with polarization values
        epochs (int): Number of training epochs (times the model tries to validate the data and recalculates its parameters)
        batch_size (int): The number of samples for every gradient update
        verbose (int): It limits how much training output is printed
        callbacks (str): It allows certain Keras callbacks (special code properties of keras). EarlyStopping, ReduceLROnPlateau or ModelCheckpoint are examples
    Outputs:
       A keras.callbacks.History object
    Note:
        It logs and runs model.fit() on a two-input Keras model and returns the training history.
        IT REQUIRES THE INPUTS (static, time and polarization) TO BE NORMALIZED/SCALED
        Training is not done using the uncertainties of the data.  
    """
    log_message(f"    Model_fitting: Training the model.")
    return model.fit(
        [X_static_scaled, X_time_scaled],
        y_scaled,
        epochs=epochs,
        batch_size=batch_size,
        verbose=verbose,
        callbacks=callbacks)


def model_prediction(model, X_static_scaled, X_time_scaled):
    """
    Arguments:
        X_static_scaled (array): A 2d array of Xs (samples, number of static features)
        X_time_scaled (array): A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        y_scaled (array): The same as X_time_scaled but with polarization values. It doesn't have to be the same as the one used in training
    Output:
        A 2d array with the predictions for those time values. It contains a column of the polarization predictions and another with the log of the variance
    Notes:
        IT REQUIRES THE INPUTS (static and time) TO BE NORMALIZED/SCALED
    """
    log_message(f"    model_prediction: Predicting {len(X_time_scaled)} time points")
    return model.predict([X_static_scaled, X_time_scaled], verbose=0)

def train(encoded_experiments, scaler_static, scaler_time, scaler_y, use_uncertainty):
    """
    Arguments:
        encoded_experiments (list): A list like this:
            encoded_experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...] 
        scaler_static: It is a scikit-learn scaler for the static features. Only MinMaxScaler (normalizes uniformly using the maximum and the minimum)
        scaler_time: Analogously to scaler_static
        scaler_y: Analogously to scaler_static
        use_uncertainty (bool): It toggles whether it uses uncertainty for predictions or not. This changes the output of the model (mean and log_var or just the mean)
    Output:
        A fully trained model that can be used by model_predictions
    Notes:
        This function is the one responsible of scaling the inputs and training the model
        1. It creates the independent arrays with all the encoded experiments (augmented or not) using build_dataset
        2. Then it scales the data. ML algorithms work better when the inputs and outputs are normalized.
           The reason why we don't normalize inside the function is to have those scaler defined globally and not locally
        3. It builds the model depending on the use_uncertainty bool. (It changes the loss function and the output).
        4. It trains the model and returns its history (the trained model)
    
    """
    log_message(f"    train: Begin training and scaling with {len(encoded_experiments)} experiments.")
    X_static_all, X_time_all, y_all, u_all = build_dataset(encoded_experiments)
    log_message(f"Scaling all data")
    X_static_scaled = scaler_static.transform(X_static_all) #It only fits to the scaler. IT DOESN'T OVERWRITE THEM. WHAT A WASTE OF 20 DAYS OF MY LIFE >:(
    X_time_scaled = scaler_time.transform(X_time_all)
    y_scaled = scaler_y.transform(y_all)
    
    model = build_model(X_static_all.shape[1], use_uncertainty=use_uncertainty)
    epochs = 300
    batch_size = 32
    log_message(f"    train: Training final model with epochs={epochs}, batch_size={batch_size} and use_uncertainty = {use_uncertainty}")
    model_fitting(model, X_static_scaled, X_time_scaled, y_scaled, epochs, batch_size, verbose=0)
    return model

############################################################

def align_static_vectors(experiments, static_columns_training, static_columns_isolated):
    """
    Arguments:
        experiments (list): A list like this:
            experiments = [
                (static_values,   # list of static parameters (per experiment)
                Deltatime,       # 1D numpy array of time values
                polarization,    # 1D numpy array of polarization values
                Uncertainty      # 1D numpy array of uncertainty values), ...] 
        static_columns_training (list): A list of strings of all the static feature names used for training
        static_columns_isolated (list): A list of strings of all the static feature names of the isolated experiment
    Output:
        aligned_experiments is a list of experiments with static vectors aligned to the training feature order.
        To be precise (aligned_static, delta_time, polarization, uncertainty) where on the parameters where static_columns_isolated
        had no values get turned into zeros. So, if static_columns_isolated doesn´t have a parameter 'Parameter 6', then, in the
        original experiments lists, all numbers associated to 'Parameter 6' get turned to zero

    Notes:
        It helps the ML algorithm to focus on the important parameters
    """
    
    aligned_experiments = []
    for static, delta_time, polarization, uncertainty in experiments:
        static_dict = dict(zip(static_columns_isolated, static))
        aligned_static = [static_dict.get(col, 0.0) for col in static_columns_training]
        aligned_experiments.append((aligned_static, delta_time, polarization, uncertainty))
    log_message(f"    align_static_vectors: Vectors aligned")
    return aligned_experiments

def model_predict_sloped(model, X_static_scaled, X_time_scaled, m, n, scaler_y, scaler_time):
    """
    Arguments:
        model: A trained model
        X_static_scaled (array). A 2d array of Xs (samples, number of static features)
        X_time_scaled (array). A 2d array of shape that reshapes Xt scaled from an array of lists (of size 1 like np.array([1],[13],[16],...)) to a 2D array
        m (float): The slope used for correction
        n (float): The polarization shift for correction
        scaler_time: It is a scikit-learn scaler for the time arrays. Only MinMaxScaler (normalizes uniformly using the maximum and the minimum)
        scaler_y: Analogously to scaler_static
    Outputs
        y_pred_corrected_scaled is a 2D array where each column is the corrected predicted values (scaled) and the second one is the log of the variance (unchanged, a.k.a scaled)
    Notes:
        1. It predicts the polarization values
        2. It inverse transforms the predicted values and the time points (not the uncertainty)
        3. It calculates the correction curve in real units and corrects the predicted values.
        4. Finally, it scales back the corrected polarization values and joins them with the unchanged logarithm of the variance
        In summary, it substracts a linear function to the predicted values. If done correctly this makes it so that the polarization
        predictions at the time points stored as static features are the same as the measured polarization values at those times.
        This fixes a vertical shift and also an overall slope. As it is a correction done with experimental values, the algorithm is still
        universal. However we can't fully say that it is a pure ML algorithm.
    """
    
    log_message(f"    model_predict_sloped: Correcting {len(X_time_scaled)} points by subtracting P(t) = {float(np.squeeze(m)):.3e} * t + {float(np.squeeze(n)):.3e}")    # Step 1: Get scaled predictions from model
    y_pred_scaled = model_prediction(model, X_static_scaled, X_time_scaled)
    mean_scaled = y_pred_scaled[:, 0:1]  # shape (N,1)
    mean_real = scaler_y.inverse_transform(mean_scaled)  # shape (N,1)
    time_real = scaler_time.inverse_transform(X_time_scaled)  # shape (N,1)
    
    correction = m * time_real + n  # shape (N,1)
    mean_corrected_real = mean_real - correction  # shape (N,1)
    
    mean_corrected_scaled = scaler_y.transform(mean_corrected_real)  # shape (N,1)
    log_var_scaled = y_pred_scaled[:, 1:2]  # shape (N,1)
    y_pred_corrected_scaled = np.hstack([mean_corrected_scaled, log_var_scaled])  # shape (N,2)
    
    return y_pred_corrected_scaled

## 6. Model creation

First it prepares the folder structure and the folder where the model will be stored.
It then loops (if told to) over all polarization columns to create models for both.
It extracts all information from the data base, scales it and saves those scalers
Finally it trains the model and saves it in a distinct folder. That folder contains the scalers (in order to decipher how to scale it back to physical units) and the trained model.

In [6]:
def Full_model_training(data_dir, model_type, output_folder, use_uncertainty, num_augmentations, build_model):
    """
    Arguments:
        data_dir (str): The directory folder that contains all the files for the data base
        model_type (str): A string with the name of the model
        use_uncertainty (bool): A bool variable that can toggle whether the model uses or not uncertainties in their output (not during training)
        num_augmentations (int): The number of augmentations used in the model.
        build_model (function): It uses the model defined from Define_Complexity. That function gets called outside this function but the build_model 
        function needs to be given as an argument for isolate_experiments
    
    Outputs:
        None
    Notes:
        First it prepares the folder structure and the folder where the model will be stored.
        It then loops (if told to) over all polarization columns to create models for both.
        It extracts all information from the data base, scales it and saves those scalers
        Finally it trains the model and saves it
    """
    #1. Prepare folders and subfolders for the newly created model
    data_dir = Path(data_dir)
    output_folder = Path(output_folder)
    output_subfolder = output_folder / f"Model_{model_type}_{num_augmentations}"
    output_subfolder.mkdir(exist_ok=True)
    log_message(f"Full_model_training: Created folder: {output_subfolder}")

    # 2. Train and predict using both modes or whatever mode you choose
    for mode in ["PolarizationD3"]: 
        # 3. Load all experiments and augment them
        log_message(f"Full_model_training: Load all experiments and augment them {num_augmentations} times.")            

        experiments, static_columns = load_all_experiments(data_dir, polarization_column=mode)
        encoded_experiments = augment_experiments(experiments, num_augmentations, base_seed=42)

        
        #4. Scalers. All polarization and time values are scaled using all the experiments. The static parameters are scaled afterwards
        log_message(f"Full_model_training: Scaling all training experiments...")
        all_time = []
        all_y = []
        
        for static, time, pol, _ in encoded_experiments:
            all_time.append(time)
            all_y.append(pol)
        
        all_time = np.concatenate(all_time).reshape(-1, 1) #unscaled
        all_y = np.concatenate(all_y).reshape(-1, 1) #unscaled
        
        scaler_time = MinMaxScaler().fit(all_time) #fit to all
        scaler_y = MinMaxScaler().fit(all_y) #fit to all
        
        # Prepare all the experiments so that they can be introduced in the training functions
        X_static_raw, X_time_raw, y_raw, u = build_dataset(encoded_experiments, mode=mode)
        scaler_static = MinMaxScaler().fit(X_static_raw) #fit to all

        if LogNoise:
            log_message(f"      Scaler static min_: , {scaler_static.min_}")
            log_message(f"      Scaler static scale_: , {scaler_static.scale_}")
            log_message(f"      Scaler static data_min_: , {scaler_static.data_min_}")
            log_message(f"      Scaler static data_max_: , {scaler_static.data_max_}")
            
            log_message(f"      Scaler time min_: , {scaler_time.min_}")
            log_message(f"      Scaler time scale_: , {scaler_time.scale_}")
            log_message(f"      Scaler time data_min_: , {scaler_time.data_min_}")
            log_message(f"      Scaler time data_max_: , {scaler_time.data_max_}")
            
            log_message(f"      Scaler polarization min_: , {scaler_y.min_}")
            log_message(f"      Scaler polarization scale_: , {scaler_y.scale_}")
            log_message(f"      Scaler polarization data_min_: , {scaler_y.data_min_}")
            log_message(f"      Scaler polarization data_max_: , {scaler_y.data_max_}")
        
        # Save the scalers. This is useful because otherwise we have no way to use the model outside of the loop (because we don't know how to unscale the values)
        joblib.dump(scaler_static, long_path(output_subfolder / f"scaler_static_{mode}.pkl"))
        joblib.dump(scaler_time, long_path(output_subfolder / f"scaler_time_{mode}.pkl"))
        joblib.dump(scaler_y, long_path(output_subfolder / f"scaler_y_{mode}.pkl"))
        log_message(f"Full_model_training: Saved scalers for mode {mode} to: {output_subfolder}")

        #5. Train and save the model on the augmented experiments. The experiments enter unscaled and get scaled inside)
        log_message(f"Full_model_training: Train the model")
        model = train(
            encoded_experiments=encoded_experiments,  
            scaler_static=scaler_static,
            scaler_time=scaler_time,
            scaler_y=scaler_y,
            use_uncertainty=use_uncertainty)


        model_path = output_subfolder / f"Model_{model_type}_{num_augmentations}.keras"
        model.save(long_path(model_path))
        log_message(f"Full_model_training: Saved model to {model_path}")

## 7 Main Code

By changing the lists Complexity and num_augmentations you can pick what model you create. Just write on the two lists your model names and the number of augmentations and it will create models combining them 
As an example,

Names = ["ModelA","ModelB","ModelC"]

Augmentations = [5,9,2]

will create the models "ModelA_5","ModelB_9" and "ModelC_2"

In [7]:
Names = ["Naif8332","Naif854","SimpleNoDropout1648"]
Augmentations = [5,6,7]
use_uncertainty = True


# Define your paths
data_dir = Path("../FileReadingStoring/AmorphousMLDataBase/").resolve()
log_file = Path.cwd().resolve() / "AmorphousExecution_times.txt"
log_file.parent.mkdir(parents=True, exist_ok=True)
os.makedirs(long_path(data_dir), exist_ok=True)

for i in range (0,len(Names)):
    start_time = time.time()
    print(f"Creating model {Names[i]}_{Augmentations[i]}")
    log_message(f"Creating model {Names[i]}_{Augmentations[i]}")
    build_model = Define_Complexity(Names[i])
    output_folder = Path("..") / "ML" / f"AmorphousModels"
    output_folder.mkdir(exist_ok=True)
    Full_model_training(
        data_dir=long_path(data_dir),
        model_type=Names[i],
        output_folder=long_path(output_folder),
        use_uncertainty=use_uncertainty,
        num_augmentations=Augmentations[i],
        build_model=build_model
    )
    end_time = time.time()
    elapsed_time = end_time - start_time
    with open(long_path(log_file), "a") as f:
        f.write(f"Execution time={elapsed_time:.6f} seconds for Crystal {Names[i]} {Augmentations[i]}\n")
    log_message(f"Execution finished in {elapsed_time:.6f} seconds. Logged to {log_file}")
    log_message(f"\n _______________________________________________________ \n ")
    plt.close('all')

Creating model Naif8332_5
Creating model Naif8332_5
Define_Complexity: Model built

Full_model_training: Created folder: \\?\C:\Users\gopeb\Desktop\D3MLPolarization-final\D3MLPolarization-main\CreateModels\ML\AmorphousModels\Model_Naif8332_5
Full_model_training: Load all experiments and augment them 5 times.
    load_experiments: Finding all Array Files...
    load_experiments: Create combined DataFrame for static parameters...
    load_experiments: Collected static data:
    load_experiments: Static dataframe columns:, ['CellID', 'AnalyserID', 'PolariserPressure', 'AnalyserPressure', 'LabPolarization', 'LabTime']
    load_experiments: Static dataframe shape:, (25, 6)
    load_experiments: Hot encoding CellID.
    load_experiments: Loaded 25 experiments.
    augment_experiments: Augmenting 25 a number of 5 times
    augment_experiments: Augmented to 150 experiments
Full_model_training: Scaling all training experiments...
    build_dataset: Starting build_dataset for column Polarization

In [3]:
for Complexity in ["Naif8108"]: 
    build_model = Define_Complexity(Complexity)
    print(Complexity)
    for num_augmentations in [5]:  # idem with augmentations
        log_message(f"Begin {Complexity} model with {num_augmentations} augmentations")
        # Delete folder if exists
        # Move all files back
        # Path to the log file
        log_file = Path("AmorphousExecution_times.txt")
        
        # Start timing
        start_time = time.time()
       
        # Recreate output folder
        output_folder = Path(f"AmorphousModels").resolve()
        output_folder.mkdir(exist_ok=True)
    
        # Run main function
        use_uncertainty = True


        isolate_experiments(
            data_dir=win_long_path(data_dir),
            model_type=model_type,
            output_folder=win_long_path(output_folder),
            use_uncertainty=use_uncertainty,
            num_augmentations=num_augmentations,
            build_model=build_model
        )

        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # Ensure the parent directory exists
        log_file.parent.mkdir(parents=True, exist_ok=True)
        
        # Open with Windows long path safety
        with open(win_long_path(str(log_file)), "a") as f:
            f.write(f"Execution time={elapsed_time:.6f} seconds for Amorphous {Complexity} {num_augmentations}\n")
        
        log_message(
            f"Execution finished in {elapsed_time:.6f} seconds "
            f"(Complexity={Complexity}, Augmentations={num_augmentations}). "
            f"Logged to {log_file}"
        )
        log_message(f"\n _______________________________________________________ \n")







Built model Naif8108
Naif8108
Begin Naif8108 model with 5 augmentations
Created folder: \\?\C:\Users\gopeb\Desktop\PolarizationProject\CreateModels\ML\AmorphousModels\Model_Naif8108_5
Finding all Array Files...
Create combined DataFrame for static parameters...
Collected static data:
Static dataframe columns: ['CellID', 'AnalyserID', 'PolariserPressure', 'AnalyserPressure', 'LabPolarization', 'LabTime']
Static dataframe shape: (25, 6)
Hot encoding CellID PLACEHOLDER, VISIT TO THE LAB MAY BE NEEDED
Loaded 25 experiments.
Augmenting 25 a number of 5 times
Augmented to 150 experiments
Scale all experiments
Starting build_dataset for column PolarizationD3 
Number of experiments to process: 150 (should be (num_augmentations+1)*(number_of_experiments-1)
Number of experiments processed for mode 'PolarizationD3': 150
Final dataset shapes: Xs: (1194, 13), Xt: (1194, 1), y: (1194, 1)
Saved scalers to: \\?\C:\Users\gopeb\Desktop\PolarizationProject\CreateModels\ML\AmorphousModels\Model_Naif8108_5

In [4]:
for Complexity in ["Naif858"]: 
    build_model = Define_Complexity(Complexity)
    print(Complexity)
    for num_augmentations in [8]:  # idem with augmentations
        log_message(f"Begin {Complexity} model with {num_augmentations} augmentations")
        # Delete folder if exists
        # Move all files back
        # Path to the log file
        log_file = Path("AmorphousExecution_times.txt")
        
        # Start timing
        start_time = time.time()
       
        # Recreate output folder
        output_folder = Path(f"AmorphousModels").resolve()
        output_folder.mkdir(exist_ok=True)
    
        # Run main function
        use_uncertainty = True


        isolate_experiments(
            data_dir=win_long_path(data_dir),
            model_type=model_type,
            output_folder=win_long_path(output_folder),
            use_uncertainty=use_uncertainty,
            num_augmentations=num_augmentations,
            build_model=build_model
        )

        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # Ensure the parent directory exists
        log_file.parent.mkdir(parents=True, exist_ok=True)
        
        # Open with Windows long path safety
        with open(win_long_path(str(log_file)), "a") as f:
            f.write(f"Execution time={elapsed_time:.6f} seconds for Amorphous {Complexity} {num_augmentations}\n")
        
        log_message(
            f"Execution finished in {elapsed_time:.6f} seconds "
            f"(Complexity={Complexity}, Augmentations={num_augmentations}). "
            f"Logged to {log_file}"
        )
        log_message(f"\n _______________________________________________________ \n")







Built model Naif858
Naif858
Begin Naif858 model with 8 augmentations
Created folder: \\?\C:\Users\gopeb\Desktop\PolarizationProject\CreateModels\ML\AmorphousModels\Model_Naif858_8
Finding all Array Files...
Create combined DataFrame for static parameters...
Collected static data:
Static dataframe columns: ['CellID', 'AnalyserID', 'PolariserPressure', 'AnalyserPressure', 'LabPolarization', 'LabTime']
Static dataframe shape: (25, 6)
Hot encoding CellID PLACEHOLDER, VISIT TO THE LAB MAY BE NEEDED
Loaded 25 experiments.
Augmenting 25 a number of 8 times
Augmented to 225 experiments
Scale all experiments
Starting build_dataset for column PolarizationD3 
Number of experiments to process: 225 (should be (num_augmentations+1)*(number_of_experiments-1)
Number of experiments processed for mode 'PolarizationD3': 225
Final dataset shapes: Xs: (1791, 13), Xt: (1791, 1), y: (1791, 1)
Saved scalers to: \\?\C:\Users\gopeb\Desktop\PolarizationProject\CreateModels\ML\AmorphousModels\Model_Naif858_8
Begi

In [5]:
for Complexity in ["Naif16108"]: 
    build_model = Define_Complexity(Complexity)
    print(Complexity)
    for num_augmentations in [14]:  # idem with augmentations
        log_message(f"Begin {Complexity} model with {num_augmentations} augmentations")
        # Delete folder if exists
        # Move all files back
        # Path to the log file
        log_file = Path("AmorphousExecution_times.txt")
        
        # Start timing
        start_time = time.time()
       
        # Recreate output folder
        output_folder = Path(f"AmorphousModels").resolve()
        output_folder.mkdir(exist_ok=True)
    
        # Run main function
        use_uncertainty = True


        isolate_experiments(
            data_dir=win_long_path(data_dir),
            model_type=model_type,
            output_folder=win_long_path(output_folder),
            use_uncertainty=use_uncertainty,
            num_augmentations=num_augmentations,
            build_model=build_model
        )

        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # Ensure the parent directory exists
        log_file.parent.mkdir(parents=True, exist_ok=True)
        
        # Open with Windows long path safety
        with open(win_long_path(str(log_file)), "a") as f:
            f.write(f"Execution time={elapsed_time:.6f} seconds for Amorphous {Complexity} {num_augmentations}\n")
        
        log_message(
            f"Execution finished in {elapsed_time:.6f} seconds "
            f"(Complexity={Complexity}, Augmentations={num_augmentations}). "
            f"Logged to {log_file}"
        )
        log_message(f"\n _______________________________________________________ \n")







Built model Naif16108
Naif16108
Begin Naif16108 model with 14 augmentations
Created folder: \\?\C:\Users\gopeb\Desktop\PolarizationProject\CreateModels\ML\AmorphousModels\Model_Naif16108_14
Finding all Array Files...
Create combined DataFrame for static parameters...
Collected static data:
Static dataframe columns: ['CellID', 'AnalyserID', 'PolariserPressure', 'AnalyserPressure', 'LabPolarization', 'LabTime']
Static dataframe shape: (25, 6)
Hot encoding CellID PLACEHOLDER, VISIT TO THE LAB MAY BE NEEDED
Loaded 25 experiments.
Augmenting 25 a number of 14 times
Augmented to 375 experiments
Scale all experiments
Starting build_dataset for column PolarizationD3 
Number of experiments to process: 375 (should be (num_augmentations+1)*(number_of_experiments-1)
Number of experiments processed for mode 'PolarizationD3': 375
Final dataset shapes: Xs: (2985, 13), Xt: (2985, 1), y: (2985, 1)
Saved scalers to: \\?\C:\Users\gopeb\Desktop\PolarizationProject\CreateModels\ML\AmorphousModels\Model_Nai

In [6]:
for Complexity in ["NaifTwiceDense16116"]: 
    build_model = Define_Complexity(Complexity)
    print(Complexity)
    for num_augmentations in [12]:  # idem with augmentations
        log_message(f"Begin {Complexity} model with {num_augmentations} augmentations")
        # Delete folder if exists
        # Move all files back
        # Path to the log file
        log_file = Path("AmorphousExecution_times.txt")
        
        # Start timing
        start_time = time.time()
       
        # Recreate output folder
        output_folder = Path(f"AmorphousModels").resolve()
        output_folder.mkdir(exist_ok=True)
    
        # Run main function
        use_uncertainty = True


        isolate_experiments(
            data_dir=win_long_path(data_dir),
            model_type=model_type,
            output_folder=win_long_path(output_folder),
            use_uncertainty=use_uncertainty,
            num_augmentations=num_augmentations,
            build_model=build_model
        )

        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # Ensure the parent directory exists
        log_file.parent.mkdir(parents=True, exist_ok=True)
        
        # Open with Windows long path safety
        with open(win_long_path(str(log_file)), "a") as f:
            f.write(f"Execution time={elapsed_time:.6f} seconds for Amorphous {Complexity} {num_augmentations}\n")
        
        log_message(
            f"Execution finished in {elapsed_time:.6f} seconds "
            f"(Complexity={Complexity}, Augmentations={num_augmentations}). "
            f"Logged to {log_file}"
        )
        log_message(f"\n _______________________________________________________ \n")







Built model NaifTwiceDense16116
NaifTwiceDense16116
Begin NaifTwiceDense16116 model with 12 augmentations
Created folder: \\?\C:\Users\gopeb\Desktop\PolarizationProject\CreateModels\ML\AmorphousModels\Model_NaifTwiceDense16116_12
Finding all Array Files...
Create combined DataFrame for static parameters...
Collected static data:
Static dataframe columns: ['CellID', 'AnalyserID', 'PolariserPressure', 'AnalyserPressure', 'LabPolarization', 'LabTime']
Static dataframe shape: (25, 6)
Hot encoding CellID PLACEHOLDER, VISIT TO THE LAB MAY BE NEEDED
Loaded 25 experiments.
Augmenting 25 a number of 12 times
Augmented to 325 experiments
Scale all experiments
Starting build_dataset for column PolarizationD3 
Number of experiments to process: 325 (should be (num_augmentations+1)*(number_of_experiments-1)
Number of experiments processed for mode 'PolarizationD3': 325
Final dataset shapes: Xs: (2587, 13), Xt: (2587, 1), y: (2587, 1)
Saved scalers to: \\?\C:\Users\gopeb\Desktop\PolarizationProject\C